In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
website = os.path.basename(os.path.dirname(os.path.realpath('__file__')))

In [ ]:
import sys
sys.path.insert(0, os.path.join(os.path.dirname(os.path.realpath('__file__')),'..'))

In [ ]:
import urllib, time
from cache import Cache
from visiondirect.scraper import ProductScraper, ReviewScraper
from config import CSV_SEPARATOR, DT_FORMAT, URL_SHORTENER
from helpers import create_download_link
import pandas as pd
from pathlib import Path

In [ ]:
home_page = 'https://www.visiondirect.es'
path = 'marcas/alcon-lentillas-soluciones/'
url = urllib.parse.urljoin(home_page, path)
parsed_url = urllib.parse.urlparse(home_page)
cache = Cache(expire_time=604800)
headers = {'X-Requested-With': 'XMLHttpRequest',
           'Authority': parsed_url.netloc,
           'Method': 'GET',
           'Schema': parsed_url.scheme,
           'Accept': 'application/json, text/javascript, */*; q=0.01', 
           'Accept-Encoding': 'gzip, deflate, br', 
           'Accept-Language': 'es-ES,es;q=0.9', 
           'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
           'Cookie': 'windowMngr={"popup":[]}; _gcl_au=1.1.955027986.1665481154; optimizelyEndUserId=oeu1665481153698r0.43681185182641147; _gid=GA1.2.660853205.1665481154; frontend=iekn2fd69imbqeit86hk6u0bm2; frontend_cid=peREF6RhI8MohKEK; tms_VisitorID=93stc0h6xh; _fbp=fb.1.1665481153967.2069137791; _pin_unauth=dWlkPU1UQTJaVEE1TURrdE56RXpOQzAwTnpNMkxUazNPREV0TkdOaVlUVmlZelJrTjJRMw; _clck=1if8k37|1|f5m|0; tms_wsip=1; _uetsid=911c5eb0494811ed8cd093e28f281d70; _uetvid=911c8d50494811ed8063154632619511; _ga=GA1.2.1693234919.1665481154; _clsk=nsqyk4|1665484842066|11|1|j.clarity.ms/collect; _ga_KD0GH5837J=GS1.1.1665484841.1.1.1665485049.0.0.0',
           'Referer': 'https://www.visiondirect.es/dailies-aquacomfort-plus', 
           'Sec-Fetch-Dest': 'empty', 
           'Sec-Fetch-Mode': 'cors', 
           'Sec-Fetch-Site': 'same-origin', 
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.5195.127 Safari/537.36'
}
parser = 'lxml'

In [ ]:
s = ProductScraper(url, cache, headers, parser)
s.run(f'Scraping {URL_SHORTENER.short(url)} ')

In [ ]:
df = s.get_data()
df.drop(['html'], axis=1).head(3)

In [ ]:
dfs = []
for _, item in df.iterrows():
    path = f'product-review/ajax/getReviewsByProductHtml?productId={item.product_id}&pageSize={item.review_count}'#'&pageNumber=1&_=1665484417232'
    headers['path'] = path
    url = urllib.parse.urljoin(f'{parsed_url.scheme}://{parsed_url.netloc}/', path)           
    s = ReviewScraper(url, cache, headers, parser)
    s.run(item.url, f'Scraping {URL_SHORTENER.short(item.url)} ')
    df = s.get_data()
    dfs.append(df)

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.concat(dfs)
df.drop('html', axis=1).head(3)

In [ ]:
output_file = f'{website}_{time.strftime(DT_FORMAT)}.csv'
create_download_link(df, filename=output_file, separator=CSV_SEPARATOR)

In [ ]:
output_file = str(Path.home() / "Downloads" / output_file)
pd.read_csv(output_file, engine='python', sep=CSV_SEPARATOR).drop('html', axis=1).sample(5).style.hide()